In [5]:
# Import Libraries
import pandas as pd
from difflib import get_close_matches
import unicodedata
import os

In [6]:
# Cargar Datos

geo = pd.read_csv("raw_data/listado-longitud-latitud-municipios-espana.csv", sep=',', dtype=str)
pobl = pd.read_csv("raw_data/poblacion.csv", sep=',', dtype=str)

pobl["2024.00"] = pobl["2024.00"].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
pobl["2024.00"] = pd.to_numeric(pobl["2024.00"], errors='coerce')

# Verificar columnas
print("Geo:", geo.columns)
print("Pobl:", pobl.columns)

Geo: Index(['Población', 'Latitud', 'Longitud', 'Altitud'], dtype='object')
Pobl: Index(['cod_municipio', 'Población', '2024.00'], dtype='object')


In [7]:
# Renombrar columnas para homogeneizar
geo = geo.rename(columns={
    "Población": "municipio",
    "Latitud": "latitud",
    "Longitud": "longitud",
    "Altitud": "altitud"
})

pobl = pobl.rename(columns={
    "Población": "municipio",
    "2024.00": "poblacion",
    "cod_municipio": "cod_municipio"
})

# Convertir la población a número 
pobl["poblacion"] = pd.to_numeric(pobl["poblacion"], errors="coerce")

# Estandarizar los nombres (mayúsculas, sin tildes, sin espacios)
def normalizar(nombre):
    nombre = str(nombre).upper().strip()
    nombre = ''.join(
        c for c in unicodedata.normalize('NFD', nombre)
        if unicodedata.category(c) != 'Mn'
    )
    return nombre

geo["municipio_norm"] = geo["municipio"].apply(normalizar)
pobl["municipio_norm"] = pobl["municipio"].apply(normalizar)

In [8]:
# Hacer un merge (left join) y detectar los no coincidentes
merged = pd.merge(geo, pobl, on="municipio_norm", how="left", suffixes=("_geo", "_pobl"))

# Municipios sin coincidencia
no_match = merged[merged["cod_municipio"].isna()]
print(f"\nMunicipios sin coincidencia exacta: {len(no_match)}")
print(no_match["municipio_geo"].head(20))

# Sugerir coincidencias cercanas 

for m in no_match["municipio_geo"].head(10):
    similar = get_close_matches(m, pobl["municipio_norm"], n=3, cutoff=0.5) 
    if similar:
        print(f"Posible emparejamiento: {m} → {similar[0]}")
    else:
        print(f"No se encontró coincidencia cercana para: {m}")


Municipios sin coincidencia exacta: 1
69    Horcajo de la Sierra
Name: municipio_geo, dtype: object
No se encontró coincidencia cercana para: Horcajo de la Sierra


In [9]:
# Coreeción manual
correcciones = {
    "HORCAJO DE LA SIERRA": "HORCAJO DE LA SIERRA-AOSLOS"
}

geo["municipio_norm"] = geo["municipio_norm"].replace(correcciones)

merged = pd.merge(geo, pobl, on="municipio_norm", how="left", suffixes=("_geo", "_pobl"))

# Filtrar los que siguen sin coincidencia
no_match = merged[merged["cod_municipio"].isna()]

print(f"Municipios sin coincidencia después de correcciones: {len(no_match)}")
print(no_match[["municipio_geo", "municipio_norm"]])

Municipios sin coincidencia después de correcciones: 0
Empty DataFrame
Columns: [municipio_geo, municipio_norm]
Index: []


In [10]:
# Limpiar y quedarte con lo importante 
final = merged[["cod_municipio", "municipio_geo", "latitud", "longitud", "altitud", "poblacion"]]
final = final.rename(columns={"municipio_geo": "municipio"})

# Filtrar los de menos de 50 000 habitantes 
final_filtrado = final[final["poblacion"] < 50000]

# Guardar resultados a CSV
output_path = os.path.join("raw_data", "municipios_madrid_menores_50000.csv")
final_filtrado.to_csv(output_path, index=False)

print("\n CSV limpio creado:", output_path)
print(f"Total municipios menores de 50 000 hab.: {len(final_filtrado)}")


 CSV limpio creado: raw_data\municipios_madrid_menores_50000.csv
Total municipios menores de 50 000 hab.: 155
